## Documentation

To read more about the get API, visit the [docs](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-get.html).

![get_documents_docs](../images/get_documents_docs.png)

## Connect to ElasticSearch


In [2]:
import { Client } from "npm:@elastic/elasticsearch";

const client = new Client({
  node: "http://localhost:9200",
  auth: {
    apiKey: "cnhtNzRaVUJoZTlWUmNVWWJFbGg6SktrbkY4aFlUSmE5d2dwbGJCUXdEZw",
  },
});

const info = await client.info();
console.log(info);


{
  name: "ff84884348ad",
  cluster_name: "docker-cluster",
  cluster_uuid: "pv2aBcEGSgGBq5GJpCBNHQ",
  version: {
    number: "8.17.4",
    build_flavor: "default",
    build_type: "docker",
    build_hash: "c63c7f5f8ce7d2e4805b7b3d842e7e792d84dda1",
    build_date: "2025-03-20T15:39:59.811110136Z",
    build_snapshot: false,
    lucene_version: "9.12.0",
    minimum_wire_compatibility_version: "7.17.0",
    minimum_index_compatibility_version: "7.0.0"
  },
  tagline: "You Know, for Search"
}


## Create index


In [8]:
await client.indices.delete({ index: "my_index", ignore_unavailable: true });
await client.indices.create({ index: "my_index" });


{ acknowledged: true, shards_acknowledged: true, index: "my_index" }

We are using the `dummy_data.json` file to insert multiple documents in the index. We store the id of each document in the `documentIds` list.


In [9]:
import data from "../data/dummy_data.json" with { type: "json" };

const documentIds = [];
data.forEach(async (document: any) => {
  const response = await client.index({
    index: "my_index",
    document,
  });

  documentIds.push(response._id);
});


In [10]:
documentIds

[
  "gK98AJYBIA2L3KcmZ1Y4",
  "ga98AJYBIA2L3KcmZ1Y4",
  "gq98AJYBIA2L3KcmZ1Y4"
]

## Get operation

This is an example of a successful operation. If the `ID` exists in the document, the `get` operation won't return any errors.

In [11]:
await client.get({
  index: "my_index",
  id: documentIds[0],
});


{
  _index: "my_index",
  _id: "gK98AJYBIA2L3KcmZ1Y4",
  _version: 1,
  _seq_no: 1,
  _primary_term: 1,
  found: true,
  _source: {
    title: "Title 1",
    text: "This is the first sample document text.",
    createdAt: "2025-03-01"
  }
}

This example shows that the `get` operation fails when providing it with an `ID` that does not exist in the index.

In [12]:
try {
  await client.get({
    index: "my_index",
    id: "does_not_exist",
  });
} catch (error) {
  console.log(error);
}


ResponseError: {"_index":"my_index","_id":"does_not_exist","found":false}
    at SniffingTransport._request (file:///C:/Users/PC/AppData/Local/deno/npm/registry.npmjs.org/@elastic/transport/8.9.5/lib/Transport.js:543:27)
    at Object.runMicrotasks (ext:core/01_core.js:692:26)
    at processTicksAndRejections (ext:deno_node/_next_tick.ts:59:10)
    at runNextTicks (ext:deno_node/_next_tick.ts:76:3)
    at eventLoopTick (ext:core/01_core.js:185:21)
    at async file:///C:/Users/PC/AppData/Local/deno/npm/registry.npmjs.org/@elastic/transport/8.9.5/lib/Transport.js:641:32
    at async SniffingTransport.request (file:///C:/Users/PC/AppData/Local/deno/npm/registry.npmjs.org/@elastic/transport/8.9.5/lib/Transport.js:637:20)
    at async Client.GetApi [as get] (file:///C:/Users/PC/AppData/Local/deno/npm/registry.npmjs.org/@elastic/elasticsearch/8.17.1/lib/api/api/get.js:44:12)
    at async <anonymous>:2:3 {
  options: { redaction: { type: "replace", additionalKeys: [] } },
  name: "ResponseEr